In [2]:
import bs4
from dotenv import load_dotenv
from langchain import hub
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import (
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain.load import dumps, loads
from langchain_core.pydantic_v1 import BaseModel, Field

from colorama import Fore
import warnings
warnings.filterwarnings("ignore")

load_dotenv()

warnings.filterwarnings("ignore")

load_dotenv()

LANGUAGE_MODEL = "gpt-4 turbo"
llm = ChatOpenAI()


In [3]:
#### INDEXING ####
urls = [
    "http://blog.langchain.dev/deconstructing-rag",
    "https://blog.langchain.dev/enhancing-rag-based-applications-accuracy-by-constructing-and-leveraging-knowledge-graphs/",
    "https://blog.langchain.dev/graph-based-metadata-filtering-for-improving-vector-search-in-rag-applications/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

In [4]:
#### Retrieval Grader : Retrieval Evaluator ####
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(description="Documents are relevant to the question, 'yes' or 'no'")

    def get_score(self) -> str:
        """Return the binary score as a string."""
        return self.binary_score


def get_score(self) -> str:
    """Return the binary score as a string."""
    return self.binary_score

# LLM with function call 
structured_llm_grader = llm.with_structured_output(GradeDocuments)

In [5]:
# Prompt 
system_template = """You are an evaluator determining the relevance of a retrieved {documents} to a user's query {question}.If the document contains keyword(s) or semantic meaning related to the question, mark it as relevant.Assign a binary score of 'yes' or 'no' to indicate the document's relevance to the question."""

system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_message_prompt = HumanMessagePromptTemplate.from_template(
    input_variables=["documents", "question"],
    template="{question}",
)
grader_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)


In [6]:
### Question Re-writer - Knowledge Refinement ####
# Prompt 
prompt_template = """Given a user input {question}, re-write or rephrase the question to optimize the query for the model"""

system_prompt = SystemMessagePromptTemplate.from_template(prompt_template)
human_prompt = HumanMessagePromptTemplate.from_template(
    input_variables=["question"],
    template="{question}",
)
re_write_prompt = ChatPromptTemplate.from_messages(
    [system_prompt, human_prompt]
)


In [7]:
### Web Search Tool - Knowledge Searching ####
web_search_tool = TavilySearchResults(k=3) 


In [8]:
#### Generate Answer  ####
# Prompt
prompt = hub.pull("rlm/rag-prompt")


/data/rag/lib/python3.10/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [9]:
#### RETRIEVAL and GENERATION ####
def assess_retrieve_docs(query):
    retrieval_grader = grader_prompt | structured_llm_grader | get_score
    docs = retriever.get_relevant_documents(query)
    doc_txt = docs[1].page_content
    binary_score = retrieval_grader.invoke({"question": query, "documents": doc_txt})
    return binary_score, docs


def rewrite_query(query):
    question_rewriter = re_write_prompt | llm | StrOutputParser()
    return question_rewriter.invoke({"question": query})

def search_web(query):
    docs = web_search_tool.invoke({"query": query})
    web_results = "\n".join([d["content"] for d in docs])
    return Document(page_content=web_results)

def generate_answer(docs, query):
    # Chain
    rag_chain = prompt | llm | StrOutputParser()

    # Run
    return rag_chain.invoke({"context": docs, "question": query})


def query():
    """Query the model with a question and assess the relevance of retrieved documents."""

    # query and evaluate
    question = "prompt engineering?"
    binary_score, docs = assess_retrieve_docs(question)
    print("binary score:", binary_score)
    if binary_score == "no":
        print(f"{Fore.MAGENTA}Retrieval is not relevant. Searching the web...{Fore.RESET}")
        docs = search_web(question) 
    print(f"{Fore.YELLOW}Retrieval, rewriting and optmize the query...{Fore.RESET}")  
    optimized_query = rewrite_query(question)
    return generate_answer(docs, optimized_query)

In [10]:
query()

/data/rag/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


binary score: no
Retrieval is not relevant. Searching the web...
Retrieval, rewriting and optmize the query...


'Prompt engineering is a new discipline used to develop and optimize prompts for language models in various applications like question answering and arithmetic reasoning. It helps improve the performance of language models by creating specific prompts for different tasks. Prompt engineering is essential in fields like medicine and writing to enhance user experience with AI models.'